In [ ]:
import os
import json
import torch
import wandb
import random
import pandas as pd

from datetime import datetime
from itertools import combinations
from tqdm.autonotebook import tqdm
from transformers import AutoTokenizer, AutoModelForCausalLM

In [ ]:
JOB_TYPE = "Dataset_Generation"
CURR_DATE_TIME = datetime.now().strftime("%Y%m%d_%H%M%S")

In [ ]:
run = wandb.init(entity="jhu-llm-prompt-recovery", project="llm-prompt-recovery", job_type="upload-dataset", name=f"{JOB_TYPE}_{CURR_DATE_TIME}")

In [ ]:
artifact = wandb.Artifact(name="bicycle-dataset", type="original-texts")

In [ ]:


tokenizer = AutoTokenizer.from_pretrained("google/gemma-1.1-7b-it")
model = AutoModelForCausalLM.from_pretrained(
    "google/gemma-1.1-7b-it", torch_dtype=torch.bfloat16
)

input_text = "Write me a poem about Machine Learning."
input_ids = tokenizer(input_text, return_tensors="pt")

outputs = model.generate(**input_ids, max_new_tokens=50)
print(tokenizer.decode(outputs[0]))
